<a href="https://colab.research.google.com/github/edsml-dg1018/CM-assessment-202324/blob/main/Task4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Task 4: Data Assimilation with Nonlinear Compression

## Objective
The objective of this task is to use a data assimilation model in a reduced space obtained by a nonlinear compression algorithm (Autoencoder). We will integrate the satellite observation data into the model data to perform real-time adjustments and evaluate the performance.



In [ ]:
from google.colab import drive
drive.mount('/content/drive')

##Data Loading and Preprocessing


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
import time
import tensorflow as tf
from tensorflow.keras.models import load_model
import joblib

# Define file paths
background_path = 'drive/MyDrive/BDA_Assessment/Ferguson_fire_background.npy'
obs_path = 'drive/MyDrive/BDA_Assessment/Ferguson_fire_obs.npy'

# Load the datasets
background_data = np.load(background_path)
obs_data = np.load(obs_path)

# Normalize the data
scaler = StandardScaler()
background_data = scaler.fit_transform(background_data.reshape(-1, background_data.shape[-1])).reshape(background_data.shape)
obs_data = scaler.transform(obs_data.reshape(-1, obs_data.shape[-1])).reshape(obs_data.shape)

# Load the saved Autoencoder model from Task 2
autoencoder = load_model('drive/MyDrive/BDA_Assessment/autoencoder.keras')

# Function to extract the encoder part of the autoencoder
def get_encoder(autoencoder):
    encoder = tf.keras.models.Model(autoencoder.input, autoencoder.get_layer('dense').output)
    return encoder

encoder = get_encoder(autoencoder)


##Data Assimilation using Kalman Filter
We will use the Kalman Filter for data assimilation. The choice of covariance matrices R and B will be motivated, and the Kalman Filter will be applied in the reduced space obtained by the Autoencoder.


In [ ]:
# Encode background and observation data using the encoder
background_data_encoded = encoder.predict(background_data)
obs_data_encoded = encoder.predict(obs_data)

# Define the covariance matrices
observation_variance = np.var(obs_data_encoded)
R = np.eye(obs_data_encoded.shape[1]) * observation_variance  # Observation noise covariance matrix

background_variance = np.var(background_data_encoded)
B = np.eye(background_data_encoded.shape[1]) * background_variance  # Background error covariance matrix


In [ ]:
# Function to apply the Kalman Filter in the latent space
def kalman_filter(background, observation, R, B):
    # Prediction step
    x_pred = background
    P_pred = B

    # Update step
    K = P_pred @ np.linalg.inv(P_pred + R)
    x_updated = x_pred + K @ (observation - background)
    P_updated = (np.eye(K.shape[0]) - K) @ P_pred

    return x_updated

# Apply the Kalman Filter in the latent space
start_time = time.time()
assimilated_data_encoded = np.array([kalman_filter(b, o, R, B) for b, o in zip(background_data_encoded, obs_data_encoded)])
end_time = time.time()
assimilation_time = end_time - start_time

# Decode the assimilated data
assimilated_data = autoencoder.predict(assimilated_data_encoded.reshape(-1, 128))  # Assuming the latent space size is 128
assimilated_data = scaler.inverse_transform(assimilated_data).reshape(background_data.shape)


##Evaluation of Assimilation
# Calculate MSE in the latent space
mse_latent = mean_squared_error(background_data_encoded.flatten(), assimilated_data_encoded.flatten())
print(f"MSE in the latent space: {mse_latent}")

# Calculate MSE in the physical space
mse_physical = mean_squared_error(background_data.flatten(), assimilated_data.flatten())
print(f"MSE in the physical space after decompression: {mse_physical}")

# Calculate the execution time
print(f"Assimilation Execution Time: {assimilation_time} seconds")
